In [9]:
from singleton_pattern import load_dataset_loader,load_model,load_config
non_dnn_method_list = load_config.get_non_dnn_method_list()
config = load_config.get_config()

model = load_model.get_model()
print(f'model:\n------------------\n{model}\n------------------')
test_loader = load_dataset_loader.get_test_loader()
print(f'test_loader: {test_loader.loader_name}')

model:
------------------
GREEN()
------------------
test_loader: UBFC_Phys


In [6]:
import datetime
def train():
    if config['method'] in non_dnn_method_list: 
        print('non train')
        return
    start_time = datetime.datetime.now()
    train_loader = load_dataset_loader.get_train_loader()
    print(f'train_loader: {train_loader.loader_name}')
    model.train(train_loader.load_data())
    runtime = datetime.datetime.now() - start_time
    hours, remainder = divmod(runtime.total_seconds(), 3600)
    minutes, seconds = divmod(remainder, 60)
    print(f"train total time:  {int(hours)}h:{int(minutes)}m:{int(seconds)}s")
# run train
train()

non train


In [3]:
# test
model.eval()

test_df = test_loader.load_data()
X,y = model.preprocessing(test_df)
pred_y = model.forward(X)

# 评价指标


TypeError: cannot unpack non-iterable NoneType object

In [17]:
model.forward([1,2,34,2,])

TypeError: mean() received an invalid combination of arguments - got (list, dim=tuple), but expected one of:
 * (Tensor input, *, torch.dtype dtype)
 * (Tensor input, tuple of ints dim, bool keepdim, *, torch.dtype dtype, Tensor out)
 * (Tensor input, tuple of names dim, bool keepdim, *, torch.dtype dtype, Tensor out)
